### Capstone Project Notebook: Week 3 Assignment: Segmenting and Clustering Neighborhoods in Toronto

In [2]:
import pandas as pd
import numpy as np
import requests

#### Using 'pandas' to parse wikipage containing postal codes of Canada.
#### Converting the table into a DataFrame called 'TorontoPostalCodes'.

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

html = requests.get(url).content
df_list = pd.read_html(html)
TorontoPostalCodes = df_list[0]
TorontoPostalCodes

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


#### Dropping all rows where 'Borough' value has 'Not Assigned'.
#### Resting index with new sequential index.

In [4]:
TorontoPostalCodes = TorontoPostalCodes[TorontoPostalCodes.Borough !='Not assigned']
TorontoPostalCodes = TorontoPostalCodes.reset_index(drop=True)
TorontoPostalCodes

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [27]:
TorontoPostalCodes.shape

(103, 3)

#### There are 103 postal codes (rows) with an assigned Borough and respective neighbourhoods listed against each postal code.

# End of Part 1 of 3

In [5]:
!pip install pgeocode
import pgeocode # import geocoder

In [29]:
TorontoPostalCodes.columns = TorontoPostalCodes.columns.str.replace(' ', '')
dfLL = pd.DataFrame(columns = ['Latitude','Longitude'])
nomi = pgeocode.Nominatim('ca')

for n in TorontoPostalCodes.index:
    df = nomi.query_postal_code(TorontoPostalCodes.PostalCode[n])
    data = {'Latitude': df.latitude, 'Longitude':df.longitude}
    dfLL = dfLL.append(data, ignore_index=True)

TorontoPostalCodes[['Latitude','Longitude']] = dfLL[['Latitude','Longitude']]
TorontoPostalCodes

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7545,-79.3300
1,M4A,North York,Victoria Village,43.7276,-79.3148
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.6518,-79.5076
99,M4Y,Downtown Toronto,Church and Wellesley,43.6656,-79.3830
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.7804,-79.2505
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.6325,-79.4939


# End of Part 2 of 3

In [7]:
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium==0.5.0
import folium # map rendering library

print('Libraries imported.')

     |████████████████████████████████| 79 kB 9.1 MB/s  eta 0:00:01
  Created wheel for folium: filename=folium-0.5.0-py3-none-any.whl size=76240 sha256=051eee477960b8282850e5f00f397e2ab9d3d427de35a94ee37a1f30e96742a2
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/b2/2f/2c/109e446b990d663ea5ce9b078b5e7c1a9c45cca91f377080f8
Successfully built folium
Libraries imported.


In [8]:
!pip install geopy
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.6534817, -79.3839347.


## 1. Exploring only those boroughs that contain the word 'Toronto'

In [30]:
options = ['Downtown Toronto','East Toronto','West Toronto','Central Toronto']

toronto_data = TorontoPostalCodes[TorontoPostalCodes['Borough'].isin(options)].reset_index(drop=True)
toronto_data

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3783
3,M5C,Downtown Toronto,St. James Town,43.6513,-79.3756
4,M4E,East Toronto,The Beaches,43.6784,-79.2941
5,M5E,Downtown Toronto,Berczy Park,43.6456,-79.3754
6,M5G,Downtown Toronto,Central Bay Street,43.6564,-79.3860
7,M6G,Downtown Toronto,Christie,43.6683,-79.4205
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.6496,-79.3833
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.6655,-79.4378


In [31]:
# create map of Toronto city using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [11]:
CLIENT_ID = 'OCGNXAFARHWAGWZ4BPFTF22JZFP0PRAGPBVSBGNUPQRIWB30' # your Foursquare ID
CLIENT_SECRET = 'L1NTEIW4IOTRCRWH23QRHWKLKHXR41VZ5S0IWJW0SQISLN33' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: OCGNXAFARHWAGWZ4BPFTF22JZFP0PRAGPBVSBGNUPQRIWB30
CLIENT_SECRET:L1NTEIW4IOTRCRWH23QRHWKLKHXR41VZ5S0IWJW0SQISLN33


In [12]:
toronto_data.loc[0, 'Neighbourhood']

'Regent Park, Harbourfront'

In [32]:
neighborhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.6555, -79.3626.


In [14]:
latitude = 43.6555
longitude = -79.3626
radius = 500
LIMIT = 100

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=OCGNXAFARHWAGWZ4BPFTF22JZFP0PRAGPBVSBGNUPQRIWB30&client_secret=L1NTEIW4IOTRCRWH23QRHWKLKHXR41VZ5S0IWJW0SQISLN33&ll=43.6555,-79.3626&v=20180605&radius=500&limit=100'

In [34]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '6006c9dfca6cb15547e39b5b'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 23,
  'suggestedBounds': {'ne': {'lat': 43.660000004500006,
    'lng': -79.3563918719477},
   'sw': {'lat': 43.6509999955, 'lng': -79.36880812805231}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '54ea41ad498e9a11e9e13308',
       'name': 'Roselle Desserts',
       'location': {'address': '362 King St E',
        'crossStreet': 'Trinity St',
        'lat': 43.653446723052674,
        'lng': -79.3620167174383,
        'labeledLatLngs': [{'label': 'di

In [35]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [36]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Figs Breakfast & Lunch,Breakfast Spot,43.655675,-79.364503
3,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149
4,The Yoga Lounge,Yoga Studio,43.655515,-79.364955


In [37]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

23 venues were returned by Foursquare.


## 2. Exploring Neighborhoods in Troronto

In [38]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [39]:


toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

In [40]:
print(toronto_venues.shape)
toronto_venues.head()

(1523, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.6555,-79.3626,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.6555,-79.3626,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.6555,-79.3626,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
3,"Regent Park, Harbourfront",43.6555,-79.3626,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
4,"Regent Park, Harbourfront",43.6555,-79.3626,The Yoga Lounge,43.655515,-79.364955,Yoga Studio


In [41]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,94,94,94,94,94,94
"Brockton, Parkdale Village, Exhibition Place",38,38,38,38,38,38
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",14,14,14,14,14,14
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",58,58,58,58,58,58
Central Bay Street,58,58,58,58,58,58
Christie,11,11,11,11,11,11
Church and Wellesley,76,76,76,76,76,76
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,22,22,22,22,22,22


In [42]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 215 uniques categories.


In [43]:
options = ['Neighborhood']

v_data = toronto_venues[toronto_venues['Venue Category'].isin(options)].reset_index(drop=True)
v_data

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.6784,-79.2941,Upper Beaches,43.680563,-79.292869,Neighborhood
1,Central Bay Street,43.6564,-79.3860,Downtown Toronto,43.653232,-79.385296,Neighborhood
2,"Richmond, Adelaide, King",43.6496,-79.3833,Downtown Toronto,43.653232,-79.385296,Neighborhood
3,"Brockton, Parkdale Village, Exhibition Place",43.6383,-79.4301,Parkdale,43.640524,-79.432200,Neighborhood
4,Stn A PO Boxes,43.6437,-79.3787,Harbourfront,43.639526,-79.380688,Neighborhood


## 3. Analysing Each Neighborhood

In [54]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# drop neighborhood column, as it is found to be one of the venue categories
toronto_onehot = toronto_onehot.drop(columns='Neighborhood')

# copy the actual neighborhood column from venues dataframe to the first column of onehot encoded dataframe
toronto_onehot.insert(loc=0, column='Neighborhood', value=toronto_venues['Neighborhood'])
toronto_onehot

,Neighborhood,Accessories Store,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Baby Store,Bagel Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1518,"Business reply mail Processing Centre, South C...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1519,"Business reply mail Processing Centre, South C...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1520,"Business reply mail Processing Centre, South C...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1521,"Business reply mail Processing Centre, South C...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [53]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot = toronto_onehot.drop(columns='Neighborhood')
toronto_onehot.insert(loc=0, column='Neighborhood', value=toronto_venues['Neighborhood'])
toronto_onehot

,Neighborhood,Accessories Store,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Baby Store,Bagel Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1518,"Business reply mail Processing Centre, South C...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1519,"Business reply mail Processing Centre, South C...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1520,"Business reply mail Processing Centre, South C...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1521,"Business reply mail Processing Centre, South C...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [55]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Accessories Store,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Baby Store,Bagel Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Yoga Studio
0,Berczy Park,0.000000,0.010638,0.021277,0.000000,0.000000,0.000000,0.000000,0.000000,0.010638,...,0.000000,0.000000,0.00,0.010638,0.000000,0.000000,0.00,0.000000,0.000000,0.010638
1,"Brockton, Parkdale Village, Exhibition Place",0.026316,0.000000,0.026316,0.000000,0.026316,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
2,"Business reply mail Processing Centre, South C...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.071429
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.000000,0.000000,0.000000,0.000000,0.017241,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.017241,0.017241
4,Central Bay Street,0.000000,0.000000,0.000000,0.017241,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.017241,0.017241,0.00,0.017241,0.000000,0.000000
5,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
6,Church and Wellesley,0.000000,0.013158,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.013158,0.00,0.000000,0.000000,0.026316
7,"Commerce Court, Victoria Hotel",0.000000,0.030000,0.010000,0.000000,0.000000,0.030000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.010000,0.000000,0.000000,0.00,0.010000,0.000000,0.000000
8,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.045455,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
9,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000


In [56]:
toronto_grouped.shape

(39, 215)

In [57]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [67]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Hotel,Seafood Restaurant,Café,Bakery,Restaurant,Japanese Restaurant,Cocktail Bar,Beer Bar,Deli / Bodega
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Thrift / Vintage Store,Gift Shop,Breakfast Spot,Italian Restaurant,Japanese Restaurant,Brewery,Hawaiian Restaurant,Sandwich Place
2,"Business reply mail Processing Centre, South C...",Restaurant,Coffee Shop,Yoga Studio,Bank,Deli / Bodega,Breakfast Spot,Bookstore,Sushi Restaurant,Japanese Restaurant,Italian Restaurant
3,"CN Tower, King and Spadina, Railway Lands, Har...",Coffee Shop,Italian Restaurant,Gym / Fitness Center,Café,Bar,New American Restaurant,Bank,Bakery,Park,French Restaurant
4,Central Bay Street,Coffee Shop,Café,Sandwich Place,Pizza Place,Bubble Tea Shop,Restaurant,Italian Restaurant,Middle Eastern Restaurant,Spa,Breakfast Spot


## 4. Clustering Neighborhoods

In [68]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([3, 3, 3, 3, 3, 0, 3, 3, 3, 3], dtype=int32)

In [69]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

toronto_merged.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626,3,Coffee Shop,Breakfast Spot,Pub,Beer Store,Italian Restaurant,Electronics Store,Restaurant,Spa,Thai Restaurant,Theater
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889,3,Sushi Restaurant,Coffee Shop,Martial Arts School,Bubble Tea Shop,Ramen Restaurant,Escape Room,Burrito Place,Portuguese Restaurant,College Theater,Japanese Restaurant
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3783,3,Coffee Shop,Clothing Store,Cosmetics Shop,Middle Eastern Restaurant,Japanese Restaurant,Hotel,Café,Furniture / Home Store,Fast Food Restaurant,Diner
3,M5C,Downtown Toronto,St. James Town,43.6513,-79.3756,3,Coffee Shop,Café,Seafood Restaurant,Italian Restaurant,Gastropub,American Restaurant,Cocktail Bar,Restaurant,Clothing Store,Farmers Market
4,M4E,East Toronto,The Beaches,43.6784,-79.2941,3,Pub,Health Food Store,Trail,Bakery,Coffee Shop,Gastropub,Dumpling Restaurant,Fish & Chips Shop,Fast Food Restaurant,Farmers Market


In [72]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examining Clusters

#### Cluster 1

In [74]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Christie,Café,Grocery Store,Park,Playground,Candy Store,Baby Store,Coffee Shop,Yoga Studio,Electronics Store,Fish & Chips Shop
10,"Harbourfront East, Union Station, Toronto Islands",Harbor / Marina,Café,Park,Music Venue,Yoga Studio,Dumpling Restaurant,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant
18,Lawrence Park,Photography Studio,Park,Lawyer,Donut Shop,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
23,"North Toronto West, Lawrence Park",Playground,Gym Pool,Park,Garden,Yoga Studio,Dumpling Restaurant,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant
29,"Moore Park, Summerhill East",Thai Restaurant,Park,Gym,Grocery Store,Trail,Doner Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
33,Rosedale,Playground,Park,Grocery Store,Candy Store,Yoga Studio,Dumpling Restaurant,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant


#### Cluster 2

In [75]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,"High Park, The Junction South",Park,Residential Building (Apartment / Condo),Yoga Studio,Dumpling Restaurant,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space


#### Cluster 3

In [76]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,"Forest Hill North & West, Forest Hill Road Park",Home Service,Park,Trail,Electronics Store,Yoga Studio,Dumpling Restaurant,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market


#### Cluster 4

In [77]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Regent Park, Harbourfront",Coffee Shop,Breakfast Spot,Pub,Beer Store,Italian Restaurant,Electronics Store,Restaurant,Spa,Thai Restaurant,Theater
1,"Queen's Park, Ontario Provincial Government",Sushi Restaurant,Coffee Shop,Martial Arts School,Bubble Tea Shop,Ramen Restaurant,Escape Room,Burrito Place,Portuguese Restaurant,College Theater,Japanese Restaurant
2,"Garden District, Ryerson",Coffee Shop,Clothing Store,Cosmetics Shop,Middle Eastern Restaurant,Japanese Restaurant,Hotel,Café,Furniture / Home Store,Fast Food Restaurant,Diner
3,St. James Town,Coffee Shop,Café,Seafood Restaurant,Italian Restaurant,Gastropub,American Restaurant,Cocktail Bar,Restaurant,Clothing Store,Farmers Market
4,The Beaches,Pub,Health Food Store,Trail,Bakery,Coffee Shop,Gastropub,Dumpling Restaurant,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
5,Berczy Park,Coffee Shop,Hotel,Seafood Restaurant,Café,Bakery,Restaurant,Japanese Restaurant,Cocktail Bar,Beer Bar,Deli / Bodega
6,Central Bay Street,Coffee Shop,Café,Sandwich Place,Pizza Place,Bubble Tea Shop,Restaurant,Italian Restaurant,Middle Eastern Restaurant,Spa,Breakfast Spot
8,"Richmond, Adelaide, King",Café,Coffee Shop,Hotel,Restaurant,Gym,American Restaurant,Salad Place,Asian Restaurant,Sushi Restaurant,Steakhouse
9,"Dufferin, Dovercourt Village",Park,Bakery,Café,Smoke Shop,Bus Line,Middle Eastern Restaurant,Brazilian Restaurant,Furniture / Home Store,Bar,Bank
11,"Little Portugal, Trinity",Bar,Coffee Shop,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Asian Restaurant,Restaurant,Cocktail Bar,Yoga Studio,Skating Rink,Italian Restaurant


#### Cluster 5

In [78]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Roselawn,IT Services,Yoga Studio,Dumpling Restaurant,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space


## 6. Observations (on Cluster Characteristics)

#### 1. Cluster-4 is the largest, with 30 neighbourhoods, that are predominently food zones (coffee shops, cafes, restaurants) among its top-5 venues
#### 2. Cluster-1 is the 2nd largest with 6 neighbourhoods, that are predominently play/wellness zones (playground, park, yoga studio)
#### 3. Cluster-2 is with just one neighbourhood, that has residential buildings
#### 4. Cluster-3 is also with just one neighbourhoof, that has home-service stores as its top-most venue
#### 5. Cluster-5 is also with just one neighbourhood, that has IT-services as its top-most venue

# End of Part-3 of 3